### Dependencies

In [38]:
!pip install -r requirements.txt
# apt-get update && apt-get install ffmpeg libsm6 libxext6  -y       RUN IN TERMINAL


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Data

We need .jpeg MRI slices and .pt file with MRI masks to train the model. We place them in the mri_data directory. \
MRI: mri_data/MRI-slices/{name-of-dir-for-current-task} \
MRI masks: mri_data/MRI-masks/{name-of-file-for-current-task} 

In [85]:
!mkdir -p mri_data
!mkdir -p mri_data/MRI-slices
!mkdir -p mri_data/MRI-masks

After creating the dirs we fill them with actual data

In [86]:
source_mri_slices_dir = "../mri-slices-ext"                 #edit according to the real place of your source data
source_mri_masks_file = "../best_slices_labels_ext.pt"      #edit according to the real place of your source data

dest_mri_slices_dir = "mri_data/MRI-slices/mri-slices-ext"
dest_mri_masks_file = "mri_data/MRI-masks/best_slices_labels_ext.pt"

!cp -r {source_mri_slices_dir} {dest_mri_slices_dir}
!cp {source_mri_masks_file} {dest_mri_masks_file}

### Config

Now we have to adjust configuration file. Here we read json file with comments, edit, and write the new configuration file without any comments.

In [97]:
import jstyleson                                        #used for reading non-standard json file with c-style comments
import json

old_config_name = "inpainting_celebahq.json"
config_name = "mri-inpainting.json"                     #edit according to the name of your experiment

with open("config/" + old_config_name, "r") as file:
    conf = jstyleson.loads(file.read())

In [98]:
conf['datasets']['train']['which_dataset']['args']['data_root'] = dest_mri_slices_dir
conf['datasets']['train']['which_dataset']['args']['mask_config']['mask_mode'] = 'file'
conf['datasets']['train']['which_dataset']['args']['mask_config']['mask_file'] = dest_mri_masks_file

conf['train']['val_epoch'] = 10
conf['train']['save_checkpoint_epoch'] = 10

conf['gpu_ids'] = [2]
conf['path']['resume_state'] = "experiments/train_inpainting-mri-real-masks_240330_183821/checkpoint/10"
conf['datasets']['test']['which_dataset']['args']['data_root'] = dest_mri_slices_dir
conf['datasets']['test']['which_dataset']['args']['mask_config']['mask_mode'] = 'bbox'
conf['name'] = 'inpainting-mri-real-masks'

In [99]:
with open("config/" + config_name, 'w') as fp:
    json.dump(conf, fp, indent=2)

Also change .gitignore file in order to hide heavy folders from git

In [100]:
dirs_to_exclude = ["mri_data/\n", "experiments/\n"]

gitignore_file = ".gitignore"

with open(gitignore_file, 'r+') as file:
    lines = file.readlines()
    for dir in dirs_to_exclude:
        if dir not in lines:
            file.write(dir)

### Running the model

Now we can train the model. All results (checkpoints, generated images, ...) will be saved in the experiments dir

In [101]:
!python run.py -p train -c config/{config_name}

export CUDA_VISIBLE_DEVICES=2
run.py:28: UserWarning: You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True
  warnings.warn('You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True')
 87%|███████████████████████████████████▊     | 286/327 [01:20<00:11,  3.54it/s]
Close the Tensorboard SummaryWriter.
Traceback (most recent call last):
  File "run.py", line 92, in <module>
    main_worker(0, 1, opt)
  File "run.py", line 58, in main_worker
    model.train()
  File "/workspace/Palette-Image-to-Image-Diffusion-Models/core/base_model.py", line 45, in train
    train_log = self.train_step()
  File "/workspace/Palette-Image-to-Image-Diffusion-Models/models/model.py", line 112, in train_step
    self.optG.step()
  File "/root/miniconda3/lib/python3.8/site-packages/torch/optim/optimizer.py", line 113, in wrapper
    return func(*args, **kwargs)
  File "/root/miniconda3/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 27, in decorat

Now we should edit configuration file before testing the model in order to use the last checkpoint. Checkpoint should have the following format: "experiments/train_inpainting_celebahq_240304_171920/checkpoint/540"

In [108]:
import os

runs = sorted([log for log in os.listdir("experiments") if conf["name"] in log])
checkpoints = sorted(os.listdir(f"experiments/{runs[-1]}/checkpoint"))
checkpoint = f"experiments/{runs[-1]}/checkpoint/{checkpoints[-1].replace('_', '.').split('.')[0]}"

with open("config/" + config_name, "r") as file:
    conf = jstyleson.loads(file.read())
    conf['path']['resume_state'] = checkpoint

with open("config/" + config_name, 'w') as fp:
    json.dump(conf, fp, indent=2)

print(checkpoint)

experiments/train_inpainting-mri-real-masks_240330_190037/checkpoint/40


Then we can test the model. Results will be saved in the experiments dir as well.

In [96]:
!python run.py -p test -c config/{config_name}

export CUDA_VISIBLE_DEVICES=2
run.py:28: UserWarning: You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True
  warnings.warn('You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True')
  0%|                                        | 3/985 [02:00<10:58:29, 40.23s/it]
Close the Tensorboard SummaryWriter.
Traceback (most recent call last):
  File "run.py", line 92, in <module>
    main_worker(0, 1, opt)
  File "run.py", line 60, in main_worker
    model.test()
  File "/workspace/Palette-Image-to-Image-Diffusion-Models/models/model.py", line 178, in test
    self.output, self.visuals = self.netG.restoration(self.cond_image, y_t=self.cond_image, 
  File "/root/miniconda3/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/Palette-Image-to-Image-Diffusion-Models/models/network.py", line 98, in restoration
    y_t = self.p_sample(y_t, t, y_cond=y_cond)
  File "

### Processing the outputs

In [ ]:
!mkdir -p generations-postprocessing
!mkdir -p generations-postprocessing